In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# %%
import torch
import string

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install transformers

##BERT

In [ ]:
#from transformers import BertTokenizer, BertForMaskedLM
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertForMaskedLM.from_pretrained('bert-base-uncased')

##XLM-R

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaForMaskedLM.from_pretrained('xlm-roberta-base')

In [ ]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


In [ ]:
# Use a subset for quick experiments
#data = data[:10000]

from sklearn.model_selection import train_test_split as tts
import pandas as pd
data = pd.read_csv("data/moh-x.csv")

# Split to train, val and test
text, test_df = tts(data[["sentence", "arg1", "verb", "label"]], random_state=42, test_size=0.1)
text, val = tts(text, random_state=42, test_size=test_df.shape[0])

In [ ]:
text = text["sentence"]

In [ ]:
#text.to_csv(r'mohx.txt', header=None, index=None, sep=' ', mode='a')

In [ ]:
with open('data/mohx.txt', 'r') as fp:
    text = fp.read().split('\n')

In [ ]:
text[:5]

['" I ca n\'t buy this story ."',
 '" European children learn the breast stroke ; they often do n\'t know how to crawl ."',
 '" He shed his image as a pushy boss ."',
 '" She devoured his novels ."',
 '" Age and experience mellowed him over the years ."']

In [ ]:
inputs = tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [ ]:
inputs

{'input_ids': tensor([[     0,     44,     87,  ...,      1,      1,      1],
        [     0,     44,  28811,  ..., 235879,      6,      2],
        [     0,     44,   1529,  ...,      1,      1,      1],
        ...,
        [     0,     44,  14838,  ...,      1,      1,      1],
        [     0,     44,   4687,  ...,      1,      1,      1],
        [     0,      2,      1,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [ ]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [ ]:
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False,  True, False,  ..., False, False, False],
        [False,  True, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False,  True, False,  ...,  True, False, False],
        [False, False, False,  ..., False, False,  True]])

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [ ]:
selection[:5]

[[11], [1, 12], [1, 13, 14, 15], [17], [6, 7, 8]]

In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [ ]:
inputs.input_ids

tensor([[     0,     44,     87,  ...,      1,      1,      1],
        [     0,    103,  28811,  ..., 235879,      6,      2],
        [     0,    103,   1529,  ...,      1,      1,      1],
        ...,
        [     0,     44,  14838,  ...,      1,      1,      1],
        [     0,    103,   4687,  ...,    103,      1,      1],
        [     0,      2,      1,  ...,      1,      1,    103]])

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

XLMRobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Laye

In [ ]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 33/33 [00:05<00:00,  6.02it/s, loss=0.0586]


##Save Model

In [ ]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = 'stockholm/mlm/mohx_xlm_finetuned/'
# output_dir = './content/xlm-roberta_model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Saving model to stockholm/mlm/mohx_xlm_finetuned/


('stockholm/mlm/mohx_xlm_finetuned/tokenizer_config.json',
 'stockholm/mlm/mohx_xlm_finetuned/special_tokens_map.json',
 'stockholm/mlm/mohx_xlm_finetuned/sentencepiece.bpe.model',
 'stockholm/mlm/mohx_xlm_finetuned/added_tokens.json')

##Import Model

In [ ]:
!pip install transformers

from transformers import XLMRobertaForSequenceClassification

output_dir = 'stockholm/mlm/mohx_xlm_finetuned/'

print(output_dir)

stockholm/mlm/mohx_xlm_finetuned/


In [ ]:
from transformers import XLMRobertaTokenizer
import torch
# Load the BERT tokenizer.
print('Loading XLMRobertaTokenizer...')
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained(output_dir)
xlmroberta_model = XLMRobertaForSequenceClassification.from_pretrained(output_dir)

Loading XLMRobertaTokenizer...


Some weights of the model checkpoint at stockholm/mlm/mohx_xlm_finetuned/ were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at stockholm/mlm/mohx_xlm_finetuned/ and are newly initialized: ['classifier.dense.b

##Masked Language Modeling Pipeline

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()

In [ ]:
from transformers import XLNetTokenizer, XLNetLMHeadModel
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
xlnet_model = XLNetLMHeadModel.from_pretrained('xlnet-base-cased').eval()

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
xlmroberta_model = XLMRobertaForMaskedLM.from_pretrained('xlm-roberta-base').eval()

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large').eval()

In [ ]:
from transformers import ElectraTokenizer, ElectraForMaskedLM
electra_tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-generator')
electra_model = ElectraForMaskedLM.from_pretrained('google/electra-small-generator').eval()

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForMaskedLM.from_pretrained('roberta-base').eval()

In [ ]:
top_k = 10

In [ ]:
def decode(tokenizer, pred_idx, top_clean):
    ignore_tokens = string.punctuation + '[PAD]'
    tokens = []
    for w in pred_idx:
        token = ''.join(tokenizer.decode(w).split())
        if token not in ignore_tokens:
            tokens.append(token.replace('##', ''))
    return '\n'.join(tokens[:top_clean])

In [ ]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
    text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
    # if <mask> is the last token, append a "." so that models dont predict punctuation.
    if tokenizer.mask_token == text_sentence.split()[-1]:
        text_sentence += ' .'

    input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
    mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    return input_ids, mask_idx

In [ ]:
def get_all_predictions(text_sentence, top_clean=5):
    # ========================= BERT =================================
    print(text_sentence)
    input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
    with torch.no_grad():
        predict = bert_model(input_ids)[0]
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLNET LARGE =================================
    input_ids, mask_idx = encode(xlnet_tokenizer, text_sentence, False)
    perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
    perm_mask[:, :, mask_idx] = 1.0  # Previous tokens don't see last token
    target_mapping = torch.zeros((1, 1, input_ids.shape[1]), dtype=torch.float)  # Shape [1, 1, seq_length] => let's predict one token
    target_mapping[0, 0, mask_idx] = 1.0  # Our first (and only) prediction will be the last token of the sequence (the masked token)

    with torch.no_grad():
        predict = xlnet_model(input_ids, perm_mask=perm_mask, target_mapping=target_mapping)[0]
    xlnet = decode(xlnet_tokenizer, predict[0, 0, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLM ROBERTA BASE =================================
    input_ids, mask_idx = encode(xlmroberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = xlmroberta_model(input_ids)[0]
    xlm = decode(xlmroberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= BART =================================
    input_ids, mask_idx = encode(bart_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = bart_model(input_ids)[0]
    bart = decode(bart_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ELECTRA =================================
    input_ids, mask_idx = encode(electra_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = electra_model(input_ids)[0]
    electra = decode(electra_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ROBERTA =================================
    input_ids, mask_idx = encode(roberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = roberta_model(input_ids)[0]
    roberta = decode(roberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    return {'bert': bert,
            'xlnet': xlnet,
            'xlm': xlm,
            'bart': bart,
            'electra': electra,
            'roberta': roberta}

In [ ]:
text_sentence = 'The stars <mask> towards each other'

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = 'The <mask> gravitate towards each other'

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = 'A hot <mask> will revive me'

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = 'A hot soup will <mask> me'

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = 'They <mask> their way to the top of the mountain .'

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = 'They clawed their <mask> to the top of the mountain .'

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = 'He revived this <mask> of opera .'

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = 'He <mask> this style of opera .'

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = 'They clawed their <mask> to the top of the mountain .'

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = 'They <mask> their way to the top of the mountain .'

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = 'We rotate the <mask> so as to maximize the use of the soil .'

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = 'We <mask> the crops so as to maximize the use of the soil .'

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "The <mask> don't harmonize ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "The colors don't <mask> ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "The <mask> is being clogged by these operations ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "The market is being <mask> by these operations ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "The <mask> was swelling with importance when she spoke of her son ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "The mother was <mask> with importance when she spoke of her son ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "Their <mask> inclines us to believe them ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "Their language <mask> us to believe them ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "capture the <mask> of Spring   ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "<mask> the essence of Spring   ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "He showered her with <mask> ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "He <mask> her with presents ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "Am I supposed to swallow that <mask> ?"

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "Am I supposed to <mask> that story ?"

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "The <mask> glared down on us ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "The sun <mask> down on us ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "<mask> erupted in the country ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "Unrest <mask> in the country ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "poison someone 's <mask> ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "<mask> someone 's mind ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "<mask> were climbing after prices were lowered ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "Sales were <mask> after prices were lowered ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "This behavior will ruin your <mask> of winning the election ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "This behavior will <mask> your chances of winning the election ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "The sales tax is <mask> into the state income tax ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "The sales <mask> is absorbed into the state income tax ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "The invaders spread their <mask> all over the country ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "The invaders <mask> their language all over the country ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "Interns have to <mask> for a few months ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "They <mask> through the job candidates ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "The dress <mask> her beautiful figure ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "<mask> the child with pride ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "The listeners <mask> when he discussed his strange ideas ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "The path <mask> all the way to the top of the hill ."

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "<mask> the contents of a book into a summary ."

In [ ]:
get_all_predictions(text_sentence)

##TroFi

In [ ]:
text_sentence = "Unfortunately , they wo n't work in the harbor tunnel where business travelers sometimes get <mask> on the way in from the airport . . . "

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "It also vindicates Pratt 's efforts to <mask> problems in customer service that contributed to its loss of dominance in the commercial jet engine field to General Electric Co"

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "Weeks later , '' Penthouse finally realized that Dominion was intentionally <mask> the deal , '' sought other financing , failed , and had to abandon the project , the judge said "

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "After years of being out of work and on welfare , many of the unemployed now lack the ability or initiative to <mask> openings "

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "She 's been able to <mask> to this policy , largely because of the nature of her music "

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "The Soviet Union is unwilling and unlikely to <mask> the deadline for withdrawing all its troops from Afghanistan , despite its latest blustering and military maneuvering there , Western and Soviet officials indicated "

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "'' I found myself <mask> out on the steps in boot camp , '' he recalls "

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "Last month , Thiokol said it settled the last suit with the estates of the seven crew members <mask> in the Challenger "

In [ ]:
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "Incumbent Spyros Kyprianou was <mask> out in voting last week"

In [ ]:
get_all_predictions(text_sentence)